### Imports

In [6]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

## Redução de linhas por modelo

## Tempo medio de cada modelo por fontes

## Precisão dos modelos por fontes

### Inferências

In [ ]:
import pandas as pd
import json
import glob
import re
from pathlib import Path

def extract_analysis_data(analysis_str):
    # Primeiro tenta com blocos markdown
    match = re.search(r"```json\s*(\{.*?\})\s*```", analysis_str, re.DOTALL)
    if not match:
        # fallback: tenta pegar qualquer JSON com CLASSIFICATION
        match = re.search(r"\{[^{}]*\"CLASSIFICATION\"[^{}]*\}", analysis_str, re.DOTALL)
    if match:
        try:
            return json.loads(match.group(1 if '```' in match.group(0) else 0))
        except json.JSONDecodeError:
            return None
    return None

# Encontrar todos os arquivos JSONL no padrão especificado
files = glob.glob('inference/**/*.jsonl', recursive=True)

data = []

for file_path in files:
    # Extrair o modelo do caminho do arquivo
    path = Path(file_path)
    model = path.parts[1]  # Extrai o segundo componente do caminho (inference/{model}/...)
    if(path.parts[2] == "data"):
        font = path.parts[2]
        type = path.parts[3]
    else:
        font = path.parts[2]
        type = path.parts[3]
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                record = json.loads(line)
                analysis_data = extract_analysis_data(record.get('analysis', ''))

                data.append({
                    'model': model,
                    'type': type,
                    'font': font,
                    'source_file': record.get('file', ''),
                    'classification': analysis_data.get('CLASSIFICATION', "NOT INTERESTING") if analysis_data else "NOT INTERESTING",
                    'confidence': analysis_data.get('CONFIDENCE', 0) if analysis_data else 0
                })

            except Exception as e:
                print(f"Erro ao processar linha do arquivo {file_path}: {str(e)}")

# Criar DataFrame
df = pd.DataFrame(data)

# Exibir estrutura do DataFrame
print("-----------------------------------")
print(f"Total de registros processados: {len(df)}")
print("\nPrimeiros registros:")
display(df)
df.to_csv("classifications.csv", index=False)

-----------------------------------
Total de registros processados: 2705

Primeiros registros:


,model,type,font,source_file,classification,confidence
0,deepseek-r1_14b,attack,data,data/attack/20250411_175749.jsonl,INTERESTING,0.70
1,deepseek-r1_14b,attack,data,data/attack/20250411_175749.jsonl,INTERESTING,0.95
2,deepseek-r1_14b,attack,data,data/attack/20250411_175749.jsonl,INTERESTING,0.85
3,deepseek-r1_14b,attack,data,data/attack/20250411_175806.jsonl,INTERESTING,0.95
4,deepseek-r1_14b,attack,data,data/attack/20250411_175826.jsonl,INTERESTING,0.95
...,...,...,...,...,...,...
2700,qwen3_14b,safe_20250505_195700.jsonl,safe,filtering/llama3.2_3b/safe/safe_20250505_19570...,NOT INTERESTING,0.60
2701,qwen3_14b,safe_20250505_195700.jsonl,safe,filtering/llama3.2_3b/safe/safe_20250505_19570...,NOT INTERESTING,0.80
2702,qwen3_14b,safe_20250505_195700.jsonl,safe,filtering/llama3.2_3b/safe/safe_20250505_19570...,NOT INTERESTING,0.00
2703,qwen3_14b,safe_20250505_200000.jsonl,safe,filtering/llama3.2_3b/safe/safe_20250505_20000...,NOT INTERESTING,0.75


### Métricas

### Gerando inferências concretas

In [8]:
import pandas as pd

def process_group(group):
    if group.empty:
        return pd.DataFrame()
    
    # Extrair metadados fixos do grupo
    model = group['model'].iloc[0]
    source_file = group['source_file'].iloc[0]
    type_ = group['type'].iloc[0]  # Assume que 'type' é consistente no grupo
    font = group['font'].iloc[0]   # Assume que 'font' é consistente no grupo

    # Contar ocorrências de cada classificação
    class_counts = group['classification'].value_counts()
    max_count = class_counts.max()
    
    # Identificar classes com contagem máxima (podem ser múltiplas em caso de empate)
    top_classes = class_counts[class_counts == max_count].index.tolist()
    
    # Critério de desempate: priorizar 'INTERESTING'
    if len(top_classes) > 1 and 'INTERESTING' in top_classes:
        majority_class = 'INTERESTING'
    else:
        majority_class = class_counts.idxmax()  # Escolhe a classe mais frequente

    # Calcular métricas consolidadas
    consolidated = pd.DataFrame({
        'model': [model],
        'font': [font],
        'source_file': [source_file],
        'type': [type_],
        'classification': [majority_class],
        'confidence': [group['confidence'].mean()],  # Média das confianças
    })
    
    return consolidated

# Carregar e processar dados
df = pd.read_csv('classifications.csv')
df['confidence'] = pd.to_numeric(df['confidence'], errors='coerce')
df = df.dropna(subset=['confidence'])

# Agrupar por modelo e arquivo, processar e consolidar
processed = (
    df.groupby(['model', 'source_file'], group_keys=False)  # Agrupamento chave
    .apply(process_group)
    .reset_index(drop=True)
)

# Ordenar colunas e salvar
final_df = processed[['model', 'font', 'source_file', 'type', 'classification', 'confidence']]
final_df.to_csv('consolidated_classifications.csv', index=False)

C:\Users\lucas\AppData\Local\Temp\ipykernel_21084\1639364311.py:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(process_group)


## Comitê

#### Voto Majoritário

In [9]:
# Configurações
COMMITTEE   = ["deepseek-r1_14b", "qwen3_14b", "mistral-nemo"]

In [10]:
INPUT_CSV   = "consolidated_classifications.csv"
OUTPUT_CSV  = "consolidated_classifications_with_vote.csv"
# 1) Leitura do CSV original
df = pd.read_csv(INPUT_CSV)

# 2) Filtra só as linhas dos 3 modelos do comitê
df_comm = df[df["model"].isin(COMMITTEE)]

# 3) Voto majoritário por source_file
teste = df_comm.groupby("source_file")["classification"]

vote = (
    df_comm
    .groupby("source_file")["classification"]
    .agg(lambda x: x.value_counts().idxmax())
)

# 4) Captura metadados (font e type) pelo primeiro registro de cada grupo
meta = (
    df_comm
    .groupby("source_file")
    .agg({
        "font": "first",
        "type": "first"
    })
)

# 5) Monta o DataFrame de saída
vote_df = pd.DataFrame({
    "model": "voto_majoritario",
    "font":       meta["font"],
    "source_file": meta.index,
    "type":       meta["type"],
    "classification": vote
}).reset_index(drop=True)

# 6) Salva em CSV
vote_df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
print(f"✅ Comitê salvo em: {OUTPUT_CSV}")

✅ Comitê salvo em: consolidated_classifications_with_vote.csv


#### Votação Ponderada por Confiança

In [11]:
def weighted_vote_classification(group: pd.DataFrame) -> str:
    """
    Recebe um grupo contendo só um source_file e retorna a classificação
    cuja soma de confiança é a maior (votação ponderada).
    """
    # Soma de confiança por label
    sums = group.groupby("classification")["confidence"].sum()
    # Retorna o índice com maior soma
    return sums.idxmax()

INPUT_CSV   = "consolidated_classifications.csv"
OUTPUT_CSV  = "consolidated_classifications_with_weighted_vote.csv"

# 1) Carrega o CSV original
df = pd.read_csv(INPUT_CSV)

# 2) Filtra só os registros dos modelos do comitê
df_comm = df[df["model"].isin(COMMITTEE)].copy()

# 3) Aplica a função de votação ponderada por grupo (source_file)
vote = (
    df_comm
    .groupby("source_file", group_keys=False)
    .apply(lambda grp: weighted_vote_classification(grp))
    .rename("classification")
)

# 4) Captura metadados (font e type) pelo primeiro registro de cada grupo
meta = (
    df_comm
    .groupby("source_file", as_index=True)
    .agg({
        "font": "first",
        "type": "first"
    })
)

# 5) Monta o DataFrame de saída com coluna `model` = "voto_ponderado"
result = pd.DataFrame({
    "model": "voto_ponderado",
    "font": meta["font"],
    "source_file": meta.index,
    "type": meta["type"],
    "classification": vote
}).reset_index(drop=True)

# 6) Salva o CSV de saída
result.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
print(f"✅ Votação ponderada salva em: {OUTPUT_CSV}")

✅ Votação ponderada salva em: consolidated_classifications_with_weighted_vote.csv


C:\Users\lucas\AppData\Local\Temp\ipykernel_21084\405435007.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda grp: weighted_vote_classification(grp))


#### Soft Voting

#### Seleção Dinâmica de Modelo

In [12]:
INPUT_CSV      = "consolidated_classifications.csv"
OUTPUT_CSV     = "consolidated_classifications_dynamic_selection.csv"

# 1) Carrega todo o CSV
df = pd.read_csv(INPUT_CSV)

# 2) Filtra apenas as linhas dos modelos do comitê
df_comm = df[df["model"].isin(COMMITTEE)].copy()

# 3) Para cada source_file, seleciona a linha com maior confidence
df_dynamic = (
    df_comm
    .sort_values(["source_file", "confidence"], ascending=[True, False])
    .groupby("source_file", as_index=False)
    .first()
    .assign(model="selecao_dinamica")
)
# 4) Salva o resultado
df_dynamic.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
print(f"✅ Seleção dinâmica salva em: {OUTPUT_CSV}")

✅ Seleção dinâmica salva em: consolidated_classifications_dynamic_selection.csv


## Resultados

### Gerando métricas finais

In [13]:
import numpy as np
INPUT_FILES = {
    "original":        "consolidated_classifications.csv",
    "dynamic":         "consolidated_classifications_dynamic_selection.csv",
    "vote":            "consolidated_classifications_with_vote.csv",
    "weighted_vote":   "consolidated_classifications_with_weighted_vote.csv",
}
OUTPUT_FILE = "all_classifications_combined.csv"

# 1) Carrega cada CSV num DataFrame
df_orig      = pd.read_csv(INPUT_FILES["original"])
df_dynamic   = pd.read_csv(INPUT_FILES["dynamic"])
df_vote      = pd.read_csv(INPUT_FILES["vote"])
df_weighted  = pd.read_csv(INPUT_FILES["weighted_vote"])

# 2) Concatena verticalmente, mantendo todas as colunas (as que não existirem em algum DF virão como NaN)
df_all = pd.concat(
    [df_orig, df_dynamic, df_vote, df_weighted],
    axis=0,
    ignore_index=True,
    sort=False
)

# 3) (Opcional) Reordena as colunas numa ordem lógica
cols_order = [
    "model", "font", "source_file", "type", "classification",
    "confidence", "false_positive", "false_negative",
    "true_positive", "true_negative"
]
# vai incluir também qualquer coluna extra que exista
cols_final = [c for c in cols_order if c in df_all.columns] \
                + [c for c in df_all.columns if c not in cols_order]
df_all = df_all[cols_final]

# 4) Salva o DataFrame combinado
df_all.to_csv(OUTPUT_FILE, index=False, encoding="utf-8")
print(f"✅ Todos os registros combinados em: {OUTPUT_FILE}")

final_df = pd.read_csv("all_classifications_combined.csv")

final_df["false_positive"] = np.where(
    (final_df["type"] == "safe") &
    (final_df["classification"] == "INTERESTING"),
    1,
    0
)

final_df["false_negative"] = np.where(
    (final_df["type"] == "attack") &
    (final_df["classification"] == "NOT INTERESTING"),
    1,
    0
)

final_df["true_positive"] = np.where(
    (final_df["type"] == "attack") &
    (final_df["classification"] == "INTERESTING"),
    1,
    0
)

final_df["true_negative"] = np.where(
    (final_df["type"] == "safe") &
    (final_df["classification"] == "NOT INTERESTING"),
    1,
    0
)

final_df.to_csv("consolidated_classifications.csv", index=False)

✅ Todos os registros combinados em: all_classifications_combined.csv


In [14]:
import pandas as pd

# Carregar o CSV
df = pd.read_csv("consolidated_classifications.csv")

# Função para calcular métricas por grupo (modelo + fonte)
def calcular_metricas(grupo):
    y_true = grupo["true_positive"] + grupo["true_negative"] > 0  # se for verdadeiro positivo ou negativo
    y_pred = ~grupo["false_positive"].astype(bool)  # predição correta se não for falso positivo

    # Isso assume que estamos interessados na predição de INTERESTING como positivo
    tp = grupo["true_positive"].sum()
    tn = grupo["true_negative"].sum()
    fp = grupo["false_positive"].sum()
    fn = grupo["false_negative"].sum()

    total = tp + tn + fp + fn

    acc = (tp + tn) / total if total else 0
    prec = tp / (tp + fp) if (tp + fp) else 0
    rec = tp / (tp + fn) if (tp + fn) else 0
    f1 = 2 * (prec * rec) / (prec + rec) if (prec + rec) else 0

    return pd.Series({
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1_score": f1
    })

# Aplicar por modelo + fonte
metricas_por_grupo = df.groupby(["model", "font"]).apply(calcular_metricas).reset_index()

# Exibir
display(metricas_por_grupo)
metricas_por_grupo.to_csv("result_table.csv", decimal=',', sep=';', index=False)


C:\Users\lucas\AppData\Local\Temp\ipykernel_21084\3569469697.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metricas_por_grupo = df.groupby(["model", "font"]).apply(calcular_metricas).reset_index()


,model,font,accuracy,precision,recall,f1_score
0,deepseek-r1_14b,attack,0.000000,0.000000,0.000000,0.000000
1,deepseek-r1_14b,data,0.816667,0.826087,0.926829,0.873563
2,deepseek-r1_14b,safe,0.000000,0.000000,0.000000,0.000000
3,llama3.1,attack,0.000000,0.000000,0.000000,0.000000
4,llama3.1,data,0.683333,0.683333,1.000000,0.811881
5,llama3.1,safe,0.000000,0.000000,0.000000,0.000000
6,mistral-nemo,attack,0.000000,0.000000,0.000000,0.000000
7,mistral-nemo,data,0.683333,0.683333,1.000000,0.811881
8,mistral-nemo,safe,0.000000,0.000000,0.000000,0.000000
9,phi4,attack,0.000000,0.000000,0.000000,0.000000


In [15]:
import pandas as pd

df = pd.read_csv("consolidated_classifications.csv")

grouped = df.groupby(["model", "font"])[
    ["false_positive", "false_negative", "true_positive", "true_negative"]
].sum().reset_index()

display(grouped)
grouped.to_csv("result_table_brute.csv", decimal=',', sep=';', index=False)

,model,font,false_positive,false_negative,true_positive,true_negative
0,deepseek-r1_14b,attack,0,0,0,0
1,deepseek-r1_14b,data,8,3,38,11
2,deepseek-r1_14b,safe,0,0,0,0
3,llama3.1,attack,0,0,0,0
4,llama3.1,data,19,0,41,0
5,llama3.1,safe,0,0,0,0
6,mistral-nemo,attack,0,0,0,0
7,mistral-nemo,data,19,0,41,0
8,mistral-nemo,safe,0,0,0,0
9,phi4,attack,0,0,0,0
